This code is not commercial and created only for study purpose. The main aim is introducing users of HPC-DA system of TU-Dresden with PyTorch and Jupyter notebook. This code does not pretend on completeness of research significance. Fell free to use and improve this code for your study purposes.

Importing PyTorch and all required libraries and checking the working capacity

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.models.resnet import ResNet, Bottleneck
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import timeit
torch.version.__version__

This is toy model that contains two linear layers. To run this model on two GPUs, simply put each linear layer on a different GPU, and move inputs and intermediate outputs to match the layer devices accordingly.

In [ ]:
class ToyModel(nn.Module):
    def __init__(self):
        super(ToyModel, self).__init__()
        self.net1 = torch.nn.Linear(10, 10).to('cuda:0')
        self.relu = torch.nn.ReLU()
        self.net2 = torch.nn.Linear(10, 5).to('cuda:1')

    def forward(self, x):
        x = self.relu(self.net1(x.to('cuda:0')))
        return self.net2(x.to('cuda:1'))

Above ToyModel looks very similar to how one would implement it on a single GPU, except the five to (device) calls which place linear layers and tensors on proper devices. The backward() and torch.optim will automatically take care of gradients as if the model is on one GPU. You only need to make sure that the labels are on the same device as the outputs when calling the loss function.

In [ ]:
model = ToyModel()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

optimizer.zero_grad()
outputs = model(torch.randn(20, 10))
labels = torch.randn(20, 5).to('cuda:1')
loss_fn(outputs, labels).backward()
optimizer.step()

Apply Model Parallel to Existing Modules.
The code below shows how to decompose torchvision.models.reset101() to two GPUs. The idea is to inherit from the existing ResNet module, and split the layers to two GPUs during construction. Then, override the forward method to stitch two sub-networks by moving the intermediate outputs accordingly.

In [ ]:
num_classes = 1000


class ModelParallelResNet101(ResNet):
    def __init__(self, *args, **kwargs):
        super(ModelParallelResNet101, self).__init__(
            Bottleneck, [3, 4, 6, 3], num_classes=num_classes, *args, **kwargs)

        self.seq1 = nn.Sequential(
            self.conv1,
            self.bn1,
            self.relu,
            self.maxpool,

            self.layer1,
            self.layer2
        ).to('cuda:0')

        self.seq2 = nn.Sequential(
            self.layer3,
            self.layer4,
            self.avgpool,
        ).to('cuda:1')

        self.fc.to('cuda:1')

    def forward(self, x):
        x = self.seq2(self.seq1(x).to('cuda:1'))
        return self.fc(x.view(x.size(0), -1))

The above implementation solves the problem for cases where the model is too large to fit into a single GPU. However, you might have already noticed that it will be slower than running it on a single GPU if your model fits. It is because, at any point in time, only one of the two GPUs are working, while the other one is sitting there doing nothing. The performance further deteriorates as the intermediate outputs need to be copied from cuda:0 to cuda:1 between layer2 and layer3.

Let us run an experiment to get a more quantitative view of the execution time. In this experiment, we train ModelParallelResNet50 and the existing torchvision.models.reset101() by running random inputs and labels through them. After the training, the models will not produce any useful predictions, but we can get a reasonable understanding of the execution times.

The train(model) method uses nn.MSELoss as the loss function, and optim.SGD as the optimizer. It mimics training on 128 X 128 images which are organized into 3 batches where each batch contains 200 images.

In [ ]:
num_batches = 3
batch_size = 200
image_w = 128
image_h = 128


def train(model):
    model.train(True)
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    one_hot_indices = torch.LongTensor(batch_size) \
                           .random_(0, num_classes) \
                           .view(batch_size, 1)

    for _ in range(num_batches):
        # generate random inputs and labels
        inputs = torch.randn(batch_size, 3, image_w, image_h)
        labels = torch.zeros(batch_size, num_classes) \
                      .scatter_(1, one_hot_indices, 1)

        # run forward pass
        optimizer.zero_grad()
        outputs = model(inputs.to('cuda:0'))

        # run backward pass
        labels = labels.to(outputs.device)
        loss_fn(outputs, labels).backward()
        optimizer.step()

Use timeit to run the train(model) method 10 times and plot the execution times with standard deviations.

In [ ]:
plt.switch_backend('Agg')


num_repeat = 10

stmt = "train(model)"

setup = "model = ModelParallelResNet101()"
# globals arg is only available in Python 3. In Python 2, use the following
# import __builtin__
# __builtin__.__dict__.update(locals())
mp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
mp_mean, mp_std = np.mean(mp_run_times), np.std(mp_run_times)

setup = "import torchvision.models as models;" + \
        "model = models.resnet101(num_classes=num_classes).to('cuda:0')"
rn_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
rn_mean, rn_std = np.mean(rn_run_times), np.std(rn_run_times)


def plot(means, stds, labels, fig_name):
    fig, ax = plt.subplots()
    ax.bar(np.arange(len(means)), means, yerr=stds,
           align='center', alpha=0.5, ecolor='red', capsize=10, width=0.6)
    ax.set_ylabel('ResNet101 Execution Time (Second)')
    ax.set_xticks(np.arange(len(means)))
    ax.set_xticklabels(labels)
    ax.yaxis.grid(True)
    plt.tight_layout()
    plt.savefig(fig_name)
    plt.close(fig)


plot([mp_mean, rn_mean],
     [mp_std, rn_std],
     ['Model Parallel', 'Single GPU'],
     'mp_vs_rn.png')
img = Image.open('mp_vs_rn.png')
img

The result shows that the execution time of model parallel implementation is longer than the existing single-GPU implementation. 
So we can conclude there is overhead in copying tensors back and forth across the GPUs. There are rooms for improvements, as we know one of the two GPUs is sitting idle throughout the execution. One option is to further divide each batch into a pipeline of splits, such that when one split reaches the second sub-network, the following split can be fed into the first sub-network. In this way, two consecutive splits can run concurrently on two GPUs.



Speed Up by Pipelining Inputs
In the following experiments, we further divide each 200-image batch into 20-image splits. As PyTorch launches CUDA operations asynchronizely, the implementation does not need to spawn multiple threads to achieve concurrency.

In [ ]:
class PipelineParallelResNet101(ModelParallelResNet101):
    def __init__(self, split_size=20, *args, **kwargs):
        super(PipelineParallelResNet101, self).__init__(*args, **kwargs)
        self.split_size = split_size

    def forward(self, x):
        splits = iter(x.split(self.split_size, dim=0))
        s_next = next(splits)
        s_prev = self.seq1(s_next).to('cuda:1')
        ret = []

        for s_next in splits:
            # A. s_prev runs on cuda:1
            s_prev = self.seq2(s_prev)
            ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

            # B. s_next runs on cuda:0, which can run concurrently with A
            s_prev = self.seq1(s_next).to('cuda:1')

        s_prev = self.seq2(s_prev)
        ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

        return torch.cat(ret)


setup = "model = PipelineParallelResNet101()"
pp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
pp_mean, pp_std = np.mean(pp_run_times), np.std(pp_run_times)

plot([mp_mean, rn_mean, pp_mean],
     [mp_std, rn_std, pp_std],
     ['Model Parallel', 'Single GPU', 'Pipelining Model Parallel'],
     'mp_vs_rn_vs_pp.png')
img = Image.open('mp_vs_rn_vs_pp.png')
img

The result shows that pipelining inputs to model parallel ResNet doesn't speed up. 
Let us introduce a new parameter 'split size' in our pipeline parallel implementation. We need to investigate how the new parameter affects the overall training time. There might be an optimal split_size configuration for this specific experiment. Let us try to find it by running experiments using several different split_size values.

In [ ]:
means = []
stds = []
split_sizes = [1, 3, 5, 8, 10, 12, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]

for split_size in split_sizes:
    setup = "model = PipelineParallelResNet101(split_size=%d)" % split_size
    pp_run_times = timeit.repeat(
        stmt, setup, number=1, repeat=num_repeat, globals=globals())
    means.append(np.mean(pp_run_times))
    stds.append(np.std(pp_run_times))

fig, ax = plt.subplots()
ax.plot(split_sizes, means)
ax.errorbar(split_sizes, means, yerr=stds, ecolor='red', fmt='ro')
ax.set_ylabel('ResNet101 Execution Time (Second)')
ax.set_xlabel('Pipeline Split Size')
ax.set_xticks(split_sizes)
ax.yaxis.grid(True)
plt.tight_layout()
plt.savefig("split_size_tradeoff.png")
plt.close(fig)
img1 = Image.open('split_size_tradeoff.png')
img1

The result shows that setting split_size to 100 achieves one of the fastest training speed. Let us see what results we will have to implement this value in to the model. Be patient. It takes time!

In [ ]:
class PipelineParallelResNet101(ModelParallelResNet101):
    def __init__(self, split_size=100, *args, **kwargs):
        super(PipelineParallelResNet101, self).__init__(*args, **kwargs)
        self.split_size = split_size

    def forward(self, x):
        splits = iter(x.split(self.split_size, dim=0))
        s_next = next(splits)
        s_prev = self.seq1(s_next).to('cuda:1')
        ret = []

        for s_next in splits:
            # A. s_prev runs on cuda:1
            s_prev = self.seq2(s_prev)
            ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

            # B. s_next runs on cuda:0, which can run concurrently with A
            s_prev = self.seq1(s_next).to('cuda:1')

        s_prev = self.seq2(s_prev)
        ret.append(self.fc(s_prev.view(s_prev.size(0), -1)))

        return torch.cat(ret)


setup = "model = PipelineParallelResNet101()"
pp_run_times = timeit.repeat(
    stmt, setup, number=1, repeat=num_repeat, globals=globals())
pp_mean, pp_std = np.mean(pp_run_times), np.std(pp_run_times)

plot([mp_mean, rn_mean, pp_mean],
     [mp_std, rn_std, pp_std],
     ['Model Parallel', 'Single GPU', 'Pipelining Model Parallel'],
     'mp_vs_rn_vs_pp.png')
img2 = Image.open('mp_vs_rn_vs_pp.png')
img2 = img2.resize((500, 300))
img2

With this parameter of the split size we could see finally improvement of the performace with using pipelining model parallel model for 2 GPUs.
However there are still opportunities to further accelerate the training process.